In [1]:
def getOriginalClass(valueMain,valueSec):
  valueMain = str(valueMain)
  valueSec = str(valueSec)

  dicClass = {
    '1':'Water',
    '2':'Ice/snow',
    '3':'Developed',
    '4':'Soil',
    '5':'Rock',
    '6':'Beach/sand',
    '7':'Deciduous',
    '8':'Evergreen',
    '9':'Mixed',
    '10':'Shrub',
    '11':'Grassland',
    '12':'Agriculture',
    '13':'Moss/lichen'
  }
  dicClassSec = {
      '4':'Barren/sparsely vegetated',
      '5':'Trees',
      '7':'Herbaceous'
  }
  if valueMain == '0':
    cls = dicClassSec[valueSec]
  else:
    cls = dicClass[valueMain]

  return cls

def getClassGPW(value):
  dicClass = {
    'Water':['Other',0],
    'Ice/snow':['Other',0],
    'Developed':['Other',0],
    'Soil':['Other',1],
    'Rock':['Other',1],
    'Beach/sand':['Other',1],
    'Deciduous':['Other',1],
    'Evergreen':['Other',1],
    'Mixed':['Other',1],
    'Shrub':['Natural and Semi-natural grass',2],
    'Grassland':['Seeded grass',3],
    'Agriculture':['Crops and other related agricultural practices',2],
    'Moss/lichen':['Natural and Semi-natural grass',1],
    'Barren/sparsely vegetated':['Other',1],
    'Trees':['Other',1],
    'Herbaceous':['not classify',0]
  }
  return dicClass[value]

def processingCluster(data,outfolder):
    chunk_size = int(data.shape[0] / 5)
    part = 1
    countyear = str(data.count_year_class.unique()[0])
    
    for start in range(0, data.shape[0], chunk_size):
        print(start,start + chunk_size)
        df_subset = data.iloc[start:start + chunk_size]
        dicrows = []
        
        for i in range(df_subset.shape[0]):
            dados = df_subset.iloc[i]
            for j in range(0,dados.count_year_class):
                rfyear = dados.Start_Year + j
                row = np.array(dados.values)
                row = np.append(row,rfyear)
                dicrows.append(row)
        
                       
        cols = list(df_subset.columns)
        cols.append('reference_year')
        
        a = gpd.GeoDataFrame(columns=cols,data=dicrows)           
        a.to_parquet(outfolder+'glance_countyear_'+countyear+'_part_'+str(part)+'.parquet')
        
        a = None
        part = part + 1
    

In [4]:
#Import modules python
import geopandas as gpd
import pandas as pd
import numpy as np
import duckdb
import warnings
warnings.filterwarnings('ignore')

In [3]:
src = '../datasets/12_Glance/bu_glance_training_dataV1.parquet'
gdf = gpd.read_parquet(src)
gdf.head(5)

,Lat,Lon,Start_Year,End_Year,Dataset_Code,Continent_Code,LC_Confidence,Segment_Type,Leaf_Type,Impervious_Percent,...,Tree_Location,Veg_Modifier,Level1_Ecoregion,Level2_Ecoregion,Glance_Class_ID_level1,Glance_Class_ID_level2,Change,ID,Glance_ID,geometry
0,1.414980,-71.083640,1985,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,5,0,False,0,02_7_00_5_0_1985_2018,POINT (-71.08364 1.41498)
1,1.972296,-70.944320,1985,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,5,0,False,1,02_7_00_5_1_1985_2018,POINT (-70.94432 1.97230)
2,2.097071,-73.839766,1985,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,7,11,False,2,02_7_00_5_2_1985_2018,POINT (-73.83977 2.09707)
3,1.205852,-71.773552,1986,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,1,1,False,3,02_7_00_5_3_1986_2018,POINT (-71.77355 1.20585)
4,1.205852,-71.775171,1986,2018,5,2,NaN,NaN,NaN,NaN,...,NaN,None,7,7_00,1,1,False,4,02_7_00_5_4_1986_2018,POINT (-71.77517 1.20585)


In [4]:
gdfDev = gdf.copy()
gdfDev['count_year_class'] = gdfDev.apply(lambda x: (x.End_Year - x.Start_Year) + 1,axis=1)
gdfDev = gdfDev[['Glance_ID','Start_Year','End_Year','count_year_class','Glance_Class_ID_level1','Glance_Class_ID_level2','geometry']]
gdfDev['dataset_name'] = 'GLanCE'
gdfDev['original_lulc_class'] = gdfDev.apply(lambda x: getOriginalClass(x.Glance_Class_ID_level2,x.Glance_Class_ID_level1),axis=1)
gdfDev['gpw_lulc_class'] = gdfDev['original_lulc_class'].apply(lambda x: getClassGPW(x)[0])
gdfDev['sample_weight'] = gdfDev['original_lulc_class'].apply(lambda x: getClassGPW(x)[1])
gdfDev['observation'] = ''

gdfDev

,Glance_ID,Start_Year,End_Year,count_year_class,Glance_Class_ID_level1,Glance_Class_ID_level2,geometry,dataset_name,original_lulc_class,gpw_lulc_class,sample_weight,observation
0,02_7_00_5_0_1985_2018,1985,2018,34,5,0,POINT (-71.08364 1.41498),GLanCE,Trees,Other,1,
1,02_7_00_5_1_1985_2018,1985,2018,34,5,0,POINT (-70.94432 1.97230),GLanCE,Trees,Other,1,
2,02_7_00_5_2_1985_2018,1985,2018,34,7,11,POINT (-73.83977 2.09707),GLanCE,Grassland,Seeded grass,3,
3,02_7_00_5_3_1986_2018,1986,2018,33,1,1,POINT (-71.77355 1.20585),GLanCE,Water,Other,0,
4,02_7_00_5_4_1986_2018,1986,2018,33,1,1,POINT (-71.77517 1.20585),GLanCE,Water,Other,0,
...,...,...,...,...,...,...,...,...,...,...,...,...
1874990,04_1_00_902_1641504_2018_2018,2018,2018,1,7,11,POINT (10.39194 46.91158),GLanCE,Grassland,Seeded grass,3,
1874991,04_1_00_902_1641505_2018_2018,2018,2018,1,7,11,POINT (9.47924 46.41012),GLanCE,Grassland,Seeded grass,3,
1874992,04_1_00_902_1641506_2018_2018,2018,2018,1,7,11,POINT (9.68595 46.42142),GLanCE,Grassland,Seeded grass,3,
1874993,04_1_00_902_1641507_2018_2018,2018,2018,1,7,11,POINT (9.95190 46.89072),GLanCE,Grassland,Seeded grass,3,


In [12]:
a = gdfDev.count_year_class.value_counts()* gdfDev.count_year_class.value_counts().index
a.sum()

31061694

In [14]:
c1 = gdfDev[gdfDev.count_year_class == 1]
c1.reset_index(inplace=True)
c1['reference_year'] = c1.End_Year
c1.to_parquet('../datasets/12_Glance/cluster2/glance_c1.parquet')

In [10]:
cgt1 = gdfDev[gdfDev.count_year_class > 1]
cgt1.reset_index(inplace=True)
listyears = np.sort(cgt1.count_year_class.unique())

In [ ]:
for i in listyears:
    processingCluster(cgt1[cgt1.count_year_class==i],'../datasets/12_Glance/cluster2/')

In [6]:
import duckdb as db

conn = db.connect()
conn.install_extension('spatial')
conn.load_extension('spatial')

In [7]:
conn.sql("SELECT * FROM 'D:/LAPIG/Doutorado/Global_Pasture_Watch/datasets/12_Glance/cluster2/*.parquet'").show()

┌────────┬──────────────────────┬────────────┬──────────┬───┬───────────────┬─────────────┬────────────────┐
│ index  │      Glance_ID       │ Start_Year │ End_Year │ … │ sample_weight │ observation │ reference_year │
│ int64  │       varchar        │   int64    │  int64   │   │     int64     │   varchar   │     int64      │
├────────┼──────────────────────┼────────────┼──────────┼───┼───────────────┼─────────────┼────────────────┤
│  28793 │ 02_30_00_701_28793…  │       2012 │     2012 │ … │             1 │             │           2012 │
│  28809 │ 02_30_00_701_28808…  │       2006 │     2006 │ … │             0 │             │           2006 │
│  28813 │ 02_29_00_701_28812…  │       2010 │     2010 │ … │             0 │             │           2010 │
│  28814 │ 02_29_00_701_28813…  │       2010 │     2010 │ … │             1 │             │           2010 │
│  28818 │ 02_30_00_701_28817…  │       2007 │     2007 │ … │             1 │             │           2007 │
│  28819 │ 02_30_00

In [8]:
conn.sql("SELECT count(*) FROM 'D:/LAPIG/Doutorado/Global_Pasture_Watch/datasets/12_Glance/cluster2/*.parquet'").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     31061694 │
└──────────────┘



In [9]:
conn.sql("DESCRIBE FROM 'D:/LAPIG/Doutorado/Global_Pasture_Watch/datasets/12_Glance/cluster2/*.parquet'").show()

┌────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name       │ column_type │  null   │   key   │ default │  extra  │
│        varchar         │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ index                  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Glance_ID              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Start_Year             │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ End_Year               │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ count_year_class       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Glance_Class_ID_level1 │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Glance_Class_ID_level2 │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ geometry               │ BLOB        │ YES     │ NULL    │ NULL    │ NULL    │
│ dataset_name           │ V

In [19]:
sql = '''
    SELECT dataset_name, reference_year, original_lulc_class, gpw_lulc_class, sample_weight, Glance_ID as observation
    FROM 'D:/LAPIG/Doutorado/Global_Pasture_Watch/datasets/12_Glance/cluster2/*.parquet'
'''
conn.sql(sql).show()

┌──────────────┬────────────────┬─────────────────────┬────────────────┬───────────────┬───────────────────────────────┐
│ dataset_name │ reference_year │ original_lulc_class │ gpw_lulc_class │ sample_weight │          observation          │
│   varchar    │     int64      │       varchar       │    varchar     │     int64     │            varchar            │
├──────────────┼────────────────┼─────────────────────┼────────────────┼───────────────┼───────────────────────────────┤
│ GLanCE       │           2012 │ Trees               │ Other          │             1 │ 02_30_00_701_28793_2012_2012  │
│ GLanCE       │           2006 │ Herbaceous          │ not classify   │             0 │ 02_30_00_701_28808_2006_2006  │
│ GLanCE       │           2010 │ Herbaceous          │ not classify   │             0 │ 02_29_00_701_28812_2010_2010  │
│ GLanCE       │           2010 │ Trees               │ Other          │             1 │ 02_29_00_701_28813_2010_2010  │
│ GLanCE       │           2007 

In [17]:
sql = '''
    COPY (
        SELECT geometry,dataset_name, reference_year, original_lulc_class, gpw_lulc_class, sample_weight, Glance_ID as observation
        FROM 'D:/LAPIG/Doutorado/Global_Pasture_Watch/datasets/12_Glance/cluster2/*.parquet'
    ) TO 'D:/LAPIG/Doutorado/Global_Pasture_Watch/datasets/harmonized_databases/parquet/from_datasets/12_db_glance_v02.parquet' (FORMAT PARQUET)
'''
conn.sql(sql)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))